In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

In [3]:
df = pd.read_csv("data.csv")
df.columns = df.columns.str.lower().str.replace(" ","_")

df["fbs"] = df["fbs"].map({True: 'true', False: 'false'})
df["exang"] = df["exang"].map({True: 'true', False: 'false'})

string = list(df.dtypes[df.dtypes == 'object'].index)

for c in string:
    df[c] = df[c].str.lower().str.replace(' ','_')
    
df["target"] = (df['num']>0).astype(int)

In [4]:
numerical = ['age',
 'trestbps',
 'chol',
 'thalch',
 'oldpeak',
 'ca'
 ]

categorical = [
 'sex',
 'cp',
 'fbs',
 'restecg',
 'exang',
 'slope',
 'thal',
]

In [5]:
del df['num']
del df['id']
del df['dataset']

In [6]:
for n in numerical:
    df[n] = df[n].fillna(df[n].median())

for n in categorical:
    df[n] = df[n].fillna(df[n].mode()[0])

In [7]:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalch      0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [8]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,target
0,63,Male,typical angina,145.0,233.0,true,lv hypertrophy,150.0,false,2.3,downsloping,0.0,fixed defect,0
1,67,Male,asymptomatic,160.0,286.0,false,lv hypertrophy,108.0,true,1.5,flat,3.0,normal,1
2,67,Male,asymptomatic,120.0,229.0,false,lv hypertrophy,129.0,true,2.6,flat,2.0,reversable defect,1
3,37,Male,non-anginal,130.0,250.0,false,normal,187.0,false,3.5,downsloping,0.0,normal,0
4,41,Female,atypical angina,130.0,204.0,false,lv hypertrophy,172.0,false,1.4,upsloping,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,54,Female,asymptomatic,127.0,333.0,true,st-t abnormality,154.0,false,0.0,flat,0.0,normal,1
916,62,Male,typical angina,130.0,139.0,false,st-t abnormality,140.0,false,0.5,flat,0.0,normal,0
917,55,Male,asymptomatic,122.0,223.0,true,st-t abnormality,100.0,false,0.0,flat,0.0,fixed defect,1
918,58,Male,asymptomatic,130.0,385.0,true,lv hypertrophy,140.0,false,0.5,flat,0.0,normal,0


In [9]:
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 1)

In [10]:
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 1)

In [11]:
len(df_train), len(df_val), len(df_test)

(552, 184, 184)

In [12]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_full_train = df_full_train.reset_index(drop=True)

In [13]:
y_train = df_train['target']
y_val = df_val['target']
y_test = df_test['target']

del df_train['target']
del df_val['target']
del df_test['target']

In [14]:
y_val

0      0
1      1
2      0
3      1
4      1
      ..
179    1
180    0
181    1
182    1
183    0
Name: target, Length: 184, dtype: int64

In [15]:
df_train

,age,sex,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal
0,67,Female,non-anginal,152.0,277.0,false,normal,172.0,false,0.0,upsloping,1.0,normal
1,44,Male,atypical angina,130.0,219.0,false,lv hypertrophy,188.0,false,0.0,upsloping,0.0,normal
2,49,Female,atypical angina,124.0,201.0,false,normal,164.0,false,0.0,flat,0.0,normal
3,45,Male,typical angina,110.0,264.0,false,normal,132.0,false,1.2,flat,0.0,reversable defect
4,52,Male,asymptomatic,130.0,225.0,false,normal,120.0,true,2.0,flat,0.0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,50,Male,asymptomatic,150.0,215.0,false,normal,140.0,true,0.0,flat,0.0,normal
548,55,Male,asymptomatic,140.0,229.0,false,normal,110.0,true,0.5,flat,0.0,normal
549,59,Male,asymptomatic,120.0,0.0,false,normal,115.0,false,0.0,flat,0.0,normal
550,45,Female,asymptomatic,138.0,236.0,false,lv hypertrophy,152.0,true,0.2,flat,0.0,normal


In [16]:
dv = DictVectorizer(sparse = False)

In [17]:
train_dicts= df_train[categorical + numerical].to_dict(orient = 'records')
val_dicts= df_val[categorical + numerical].to_dict(orient = 'records')

In [18]:
dv.fit(train_dicts)

,"dtype dtype: dtype, default=np.float64The type of feature values. Passed to Numpy array/scipy.sparse matrixconstructors as the dtype argument.",<class 'numpy.float64'>
,"separator separator: str, default=""=""Separator string used when constructing new features for one-hotcoding.",'='
,"sparse sparse: bool, default=TrueWhether transform should produce scipy.sparse matrices.",False
,"sort sort: bool, default=TrueWhether ``feature_names_`` and ``vocabulary_`` should besorted when fitting.",True


In [19]:
dv.get_feature_names_out()

array(['age', 'ca', 'chol', 'cp=asymptomatic', 'cp=atypical angina',
       'cp=non-anginal', 'cp=typical angina', 'exang=false', 'exang=true',
       'fbs=false', 'fbs=true', 'oldpeak', 'restecg=lv hypertrophy',
       'restecg=normal', 'restecg=st-t abnormality', 'sex=Female',
       'sex=Male', 'slope=downsloping', 'slope=flat', 'slope=upsloping',
       'thal=fixed defect', 'thal=normal', 'thal=reversable defect',
       'thalch', 'trestbps'], dtype=object)

In [20]:
dv.transform(train_dicts[:5])

array([[ 67. ,   1. , 277. ,   0. ,   0. ,   1. ,   0. ,   1. ,   0. ,
          1. ,   0. ,   0. ,   0. ,   1. ,   0. ,   1. ,   0. ,   0. ,
          0. ,   1. ,   0. ,   1. ,   0. , 172. , 152. ],
       [ 44. ,   0. , 219. ,   0. ,   1. ,   0. ,   0. ,   1. ,   0. ,
          1. ,   0. ,   0. ,   1. ,   0. ,   0. ,   0. ,   1. ,   0. ,
          0. ,   1. ,   0. ,   1. ,   0. , 188. , 130. ],
       [ 49. ,   0. , 201. ,   0. ,   1. ,   0. ,   0. ,   1. ,   0. ,
          1. ,   0. ,   0. ,   0. ,   1. ,   0. ,   1. ,   0. ,   0. ,
          1. ,   0. ,   0. ,   1. ,   0. , 164. , 124. ],
       [ 45. ,   0. , 264. ,   0. ,   0. ,   0. ,   1. ,   1. ,   0. ,
          1. ,   0. ,   1.2,   0. ,   1. ,   0. ,   0. ,   1. ,   0. ,
          1. ,   0. ,   0. ,   0. ,   1. , 132. , 110. ],
       [ 52. ,   0. , 225. ,   1. ,   0. ,   0. ,   0. ,   0. ,   1. ,
          1. ,   0. ,   2. ,   0. ,   1. ,   0. ,   0. ,   1. ,   0. ,
          1. ,   0. ,   0. ,   1. ,   0. , 120. , 130. ]])

In [21]:
dv.transform(val_dicts[:5])

array([[ 71. ,   2. , 302. ,   0. ,   1. ,   0. ,   0. ,   1. ,   0. ,
          1. ,   0. ,   0.4,   0. ,   1. ,   0. ,   1. ,   0. ,   0. ,
          0. ,   1. ,   0. ,   1. ,   0. , 162. , 160. ],
       [ 46. ,   0. , 249. ,   1. ,   0. ,   0. ,   0. ,   1. ,   0. ,
          1. ,   0. ,   0.8,   1. ,   0. ,   0. ,   0. ,   1. ,   0. ,
          0. ,   1. ,   0. ,   0. ,   1. , 144. , 120. ],
       [ 60. ,   0. ,   0. ,   1. ,   0. ,   0. ,   0. ,   0. ,   1. ,
          1. ,   0. ,   0. ,   0. ,   0. ,   1. ,   0. ,   1. ,   0. ,
          1. ,   0. ,   0. ,   0. ,   1. , 118. , 152. ],
       [ 60. ,   0. , 186. ,   1. ,   0. ,   0. ,   0. ,   0. ,   1. ,
          0. ,   1. ,   0.5,   1. ,   0. ,   0. ,   0. ,   1. ,   0. ,
          1. ,   0. ,   0. ,   1. ,   0. , 140. , 130. ],
       [ 57. ,   0. , 270. ,   1. ,   0. ,   0. ,   0. ,   0. ,   1. ,
          0. ,   1. ,   2. ,   0. ,   0. ,   1. ,   0. ,   1. ,   0. ,
          1. ,   0. ,   0. ,   1. ,   0. , 160. , 144. ]])

In [22]:
X_train = dv.fit_transform(train_dicts)

In [23]:
X_val = dv.transform(val_dicts)

In [24]:
model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [25]:
y_pred = model.predict_proba(X_val)[:, 1]

In [26]:
target_decision = y_pred>=0.5

In [27]:
target_decision

array([False,  True,  True,  True,  True, False,  True,  True, False,
        True, False, False, False,  True, False,  True, False,  True,
        True, False, False,  True,  True,  True, False, False, False,
        True,  True,  True,  True,  True,  True, False, False,  True,
       False, False, False, False,  True,  True,  True,  True,  True,
        True, False, False,  True, False,  True, False, False,  True,
        True,  True,  True,  True, False, False,  True, False,  True,
       False,  True, False,  True,  True, False,  True, False, False,
       False,  True,  True, False, False,  True,  True, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False,  True, False,  True, False,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True, False, False,  True,  True,  True, False,  True,  True,
       False,  True, False,  True,  True, False, False, False,  True,
       False, False,

In [28]:
(y_val == target_decision).mean()

np.float64(0.8043478260869565)

In [29]:
df_pred = pd.DataFrame()

In [31]:
df_pred['probability'] = y_pred

In [33]:
df_pred['prediction'] = target_decision.astype(int)

In [35]:
df_pred['actual'] = y_val

In [36]:
df_pred

,probability,prediction,actual
0,0.087452,0,0
1,0.509525,1,1
2,0.945822,1,0
3,0.879683,1,1
4,0.907861,1,1
...,...,...,...
179,0.707689,1,1
180,0.048146,0,0
181,0.731914,1,1
182,0.902533,1,1
